<a href="https://colab.research.google.com/github/ljtamang/codeBERT-QG/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install pacakages below:

In [2]:
!pip install torch
!pip install transformers
!pip install nltk
!pip install psutil
!pip install h5py pip install typing-extensions pip install wheel  # If this is not installed, it shows error in below blue package isntallation
!pip install blue

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!python --version

Python 3.8.10


In [ ]:
!git clone https://github.com/ljtamang/codeBERT-QG.git

Cloning into 'codeBERT-QG'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 101 (delta 17), reused 23 (delta 7), pack-reused 60
Receiving objects: 100% (101/101), 79.98 MiB | 20.83 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [ ]:
%cd codeBERT-QG
#%cd codeBERT-QG/code

/content/codeBERT-QG


In [ ]:
# check ram
#!cat /proc/meminfo

# List device allocated by google collab
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices() 

# Test if GUP is enabled
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
#when GPU 1 is available
!bash java_script.sh 0 cbQG


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
model_name_or_path: microsoft/codebert-base
tokenizer_name : 
False
roberta
None


In [ ]:
# Run the Test
!bash java_script_test.sh 0 cbQG

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Traceback (most recent call last):
  File "run.py", line 669, in <module>
    main()
  File "run.py", line 313, in main
    model.load_state_dict(torch.load(args.load_model_path))
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", line 771, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", line 270, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", line 251, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: '../output/cbQG/checkpoint-best-bleu/pytorch_model.bin'


In [ ]:
from __future__ import absolute_import
import os
import sys
import bleu
import pickle
import torch
import json
import random
import logging
import argparse
import numpy as np
from io import open
from itertools import cycle
from collections import OrderedDict, Counter
import torch.nn as nn
from model import Seq2Seq
from tqdm import tqdm, trange
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaModel, RobertaTokenizer)
from eval.bleu import corpus_bleu
from eval.rouge import Rouge
from eval.meteor import Meteor
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaModel, RobertaTokenizer)}
